# Quick Start: Sri Lanka Reserves Forecasting

This notebook provides a quick introduction to the reserves forecasting codebase.

**Contents:**
1. Load and explore the data
2. Visualize reserves and key predictors
3. Run a simple MS-VAR forecast
4. Compare with baseline models

In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

## 1. Load the Data

The main forecasting panel contains 252 monthly observations (2005-2025) with reserves and key predictors.

In [ ]:
# Load the main forecasting panel
DATA_PATH = '../data/merged/reserves_forecasting_panel.csv'
df = pd.read_csv(DATA_PATH, parse_dates=['date'], index_col='date')

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df.index.min().strftime('%Y-%m')} to {df.index.max().strftime('%Y-%m')}")
print(f"\nVariables:")
for col in df.columns:
    print(f"  - {col}")

In [ ]:
# Summary statistics
df.describe().round(2)

## 2. Visualize Reserves History

Sri Lanka's reserves show dramatic movements, particularly during the 2022 crisis.

In [ ]:
# Plot reserves history with key events
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(df.index, df['gross_reserves_usd_m'], linewidth=2, color='#2E86AB')
ax.fill_between(df.index, 0, df['gross_reserves_usd_m'], alpha=0.3, color='#2E86AB')

# Mark key events
events = {
    '2019-04-01': 'Easter Attacks',
    '2020-03-01': 'COVID-19',
    '2022-04-01': 'Sovereign Default',
    '2023-03-01': 'IMF Program'
}

for date, label in events.items():
    ax.axvline(pd.Timestamp(date), color='red', linestyle='--', alpha=0.5)
    ax.text(pd.Timestamp(date), ax.get_ylim()[1]*0.95, label, 
            rotation=90, va='top', ha='right', fontsize=9)

ax.set_xlabel('Date')
ax.set_ylabel('Gross Reserves (USD millions)')
ax.set_title('Sri Lanka Foreign Exchange Reserves (2005-2025)')
plt.tight_layout()
plt.show()

In [ ]:
# Plot key predictors
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

variables = [
    ('trade_balance_usd_m', 'Trade Balance (USD M)'),
    ('usd_lkr', 'USD/LKR Exchange Rate'),
    ('remittances_usd_m', 'Remittances (USD M)'),
    ('tourism_usd_m', 'Tourism Receipts (USD M)')
]

for ax, (var, title) in zip(axes.flat, variables):
    if var in df.columns:
        ax.plot(df.index, df[var], linewidth=1.5)
        ax.set_title(title)
        ax.set_xlabel('')

plt.tight_layout()
plt.show()

## 3. Train/Test Split

We use an expanding window evaluation:
- **Training**: 2005-2019 (180 months)
- **Validation**: 2020-2022 (36 months, includes crisis)
- **Test**: 2023-2025 (36 months, post-default)

In [ ]:
# Define splits
TRAIN_END = '2019-12-01'
VALID_END = '2022-12-01'

train = df.loc[:TRAIN_END]
valid = df.loc[TRAIN_END:VALID_END].iloc[1:]
test = df.loc[VALID_END:].iloc[1:]

print(f"Training:   {len(train)} months ({train.index.min().strftime('%Y-%m')} to {train.index.max().strftime('%Y-%m')})")
print(f"Validation: {len(valid)} months ({valid.index.min().strftime('%Y-%m')} to {valid.index.max().strftime('%Y-%m')})")
print(f"Test:       {len(test)} months ({test.index.min().strftime('%Y-%m')} to {test.index.max().strftime('%Y-%m')})")

In [ ]:
# Visualize splits
fig, ax = plt.subplots(figsize=(14, 5))

ax.plot(train.index, train['gross_reserves_usd_m'], label='Training', linewidth=2)
ax.plot(valid.index, valid['gross_reserves_usd_m'], label='Validation', linewidth=2)
ax.plot(test.index, test['gross_reserves_usd_m'], label='Test', linewidth=2)

ax.axvline(pd.Timestamp(TRAIN_END), color='gray', linestyle='--', alpha=0.7)
ax.axvline(pd.Timestamp(VALID_END), color='gray', linestyle='--', alpha=0.7)

ax.legend()
ax.set_ylabel('Gross Reserves (USD M)')
ax.set_title('Train/Validation/Test Split')
plt.tight_layout()
plt.show()

## 4. Simple MS-VAR Forecast Example

The Markov-Switching VAR is our best-performing model. Here's how to fit and forecast with it.

In [ ]:
# Import the MS-VAR model
try:
    from reserves_project.models.ms_switching_var import MarkovSwitchingVAR
    MSVAR_AVAILABLE = True
except ImportError:
    print("MS-VAR not available. Run 'pip install -e .' from the reserves_project directory.")
    MSVAR_AVAILABLE = False

In [ ]:
if MSVAR_AVAILABLE:
    # Prepare data (parsimonious variable set)
    vars_to_use = ['gross_reserves_usd_m', 'trade_balance_usd_m', 'usd_lkr']
    train_data = train[vars_to_use].dropna()
    
    # Difference the data (all variables are I(1))
    train_diff = train_data.diff().dropna()
    
    # Initialize and fit MS-VAR
    msvar = MarkovSwitchingVAR(n_regimes=2, ar_order=2)
    msvar.fit(train_diff.values)
    
    print("MS-VAR fitted successfully!")
    print(f"\nTransition Matrix:")
    print(np.round(msvar.transition_matrix_, 3))
else:
    print("Skipping MS-VAR example (package not installed)")

In [ ]:
if MSVAR_AVAILABLE:
    # Generate 12-month ahead forecast
    horizon = 12
    diff_forecast = msvar.forecast(horizon)
    
    # Convert back to levels
    last_level = train_data['gross_reserves_usd_m'].iloc[-1]
    level_forecast = last_level + np.cumsum(diff_forecast[:, 0])
    
    # Create forecast dates
    last_date = train_data.index[-1]
    forecast_dates = pd.date_range(start=last_date, periods=horizon+1, freq='MS')[1:]
    
    # Plot
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Historical
    ax.plot(train_data.index[-36:], train_data['gross_reserves_usd_m'].iloc[-36:], 
            label='Historical', linewidth=2, color='#2E86AB')
    
    # Actual (validation period)
    actual = df.loc[TRAIN_END:VALID_END, 'gross_reserves_usd_m'].iloc[1:horizon+1]
    ax.plot(actual.index, actual.values, label='Actual', linewidth=2, color='#28A745')
    
    # Forecast
    ax.plot(forecast_dates, level_forecast, label='MS-VAR Forecast', 
            linewidth=2, linestyle='--', color='#DC3545')
    
    ax.axvline(pd.Timestamp(TRAIN_END), color='gray', linestyle=':', alpha=0.7)
    ax.legend()
    ax.set_ylabel('Gross Reserves (USD M)')
    ax.set_title('MS-VAR 12-Month Ahead Forecast')
    plt.tight_layout()
    plt.show()

## 5. Load Pre-Computed Results

The full rolling-origin evaluation has already been run. Let's load and visualize the results.

In [ ]:
# Load pre-computed model comparison
results_path = '../data/forecast_results_unified/rolling_origin_summary_parsimonious.csv'

try:
    results = pd.read_csv(results_path)
    print("Loaded pre-computed results!")
    
    # Filter for h=1 test set
    h1_test = results[(results['horizon'] == 1) & (results['split'] == 'test')]
    h1_test = h1_test.sort_values('rmse')
    
    print("\n=== Model Rankings (h=1, Test Set) ===")
    print(h1_test[['model', 'rmse', 'mae', 'mape']].to_string(index=False))
except FileNotFoundError:
    print(f"Results not found at {results_path}")
    print("Run 'slreserves evaluate --include-ms' to generate results.")

In [ ]:
# Visualize model comparison
if 'results' in dir():
    h1_test = results[(results['horizon'] == 1) & (results['split'] == 'test')].sort_values('rmse')
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # RMSE comparison
    colors = ['#DC3545' if 'MS-' in m else '#6C757D' for m in h1_test['model']]
    colors = ['#28A745' if m == 'MS-VAR' else c for m, c in zip(h1_test['model'], colors)]
    
    axes[0].barh(h1_test['model'], h1_test['rmse'], color=colors)
    axes[0].set_xlabel('RMSE (USD M)')
    axes[0].set_title('Model Comparison: RMSE (h=1)')
    axes[0].invert_yaxis()
    
    # MAPE comparison  
    axes[1].barh(h1_test['model'], h1_test['mape'], color=colors)
    axes[1].set_xlabel('MAPE (%)')
    axes[1].set_title('Model Comparison: MAPE (h=1)')
    axes[1].invert_yaxis()
    
    plt.tight_layout()
    plt.show()
    
    print("\n✓ MS-VAR achieves the lowest RMSE and MAPE on the test set!")

## Next Steps

- **02_model_comparison.ipynb**: Detailed model comparison across horizons and variable sets
- **03_scenario_analysis.ipynb**: Policy scenario forecasting

Or run the full pipeline from the command line:
```bash
slreserves replicate --run-id my_replication
```